In [31]:
import numpy as np
from data import Vertebral_column, indian_liver_patient
from Processing_Data import Haberman
from Processing_Data import Waveform
from Processing_Data import New_thyroid2
from Processing_Data import Page_blocks
from Processing_Data import Pima_Indians_Diabetes, Pima
from Processing_Data import Satimage
from Processing_Data import Transfusion, Transfution_Kfold
from Processing_Data import Yeast, Yeast_KFold, Waveform_KFold, Satimage_KFold
from wsvm.application import Wsvm
from svm.application import Svm
from sklearn.svm import SVC
from sklearn.metrics  import classification_report,precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import _safe_indexing
from sklearn import metrics
from datetime import datetime
from fuzzy.weight import fuzzy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from Processing_Data import Ecoli_Kfold
from Processing_Data import Haberman_KFold
import csv
import math


In [32]:
def Find_Max(arr):
    """
    :param arr: list of integer
    :return: item max of list
    """
    max = arr[0]

    for i in range(len(arr)):
        if max < arr[i]:
            max = arr[i]

    return max


In [33]:
import pandas as pd
df = pd.read_csv('C:/Users/ADMIN/OneDrive/Desktop/KQTN-2010/Data_Yeast_KFold_19102022_204220_Full.csv')
for i in range(len(df)):
    if (df['Time'][i] == 1 and df['Fold'][i] == 1 and df['T'][i] != 0):
        if (df['Name Method'][i] == 'own_class_center' and df['Name Function'][i] == 'lin_center_own'):
            print(df['Gmean'][i])

0.624695048
0.622459995
0.624695048
0.624695048
0.624695048
0.624695048
0.624695048
0.624695048
0.624695048
0.626922131
0.626922131
0.626922131
0.626922131
0.626922131
0.626922131
0.626922131
0.626922131
0.626922131
0.626922131
0.626922131


In [34]:
def svm_lib(X_train, y_train,X_test):
    svc=SVC(probability=True, kernel='linear')
    model = svc.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [35]:
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [36]:
def svm(C,X_train, y_train,X_test):
    model = Svm(C)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [37]:
def is_tomek_new(X,y, class_type):
    print(y)
    print(type(y))
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X)
    nn_index = nn.kneighbors(X, return_distance=False)[:, 1] #Trả về một mảng gồm chỉ số của các điểm gần nhất của mỗi mẫu tương ứng
    links = np.zeros(len(y), dtype=bool)
    # find which class to not consider
    class_excluded = [c for c in np.unique(y) if c not in class_type] #[1.0]
    X_dangxet = []
    X_tl = []
    # there is a Tomek link between two samples if they are both nearest
    # neighbors of each others.
    for index_sample, target_sample in enumerate(y):
        if target_sample in class_excluded:
            continue
        if y[nn_index[index_sample]] != target_sample: #target_sample=[-1.0]; Kiểm tra nhãn của những điểm gần nhất trong mảng nn_index (điểm được xét và điểm gần nhất phải khác nhãn)
            if nn_index[nn_index[index_sample]] == index_sample:
                X_tl.append(index_sample) #X_tl: mảng chứa chỉ số của những mẫu âm trong các cặp TL
                X_dangxet.append(nn_index[index_sample]) #X_dangxet: mảng chứa chỉ số của những mẫu dương trong các cặp TL
                links[index_sample] = True #Cập nhật trạng thái của các mẫu âm trong cặp TL

    stt = np.zeros(len(X_dangxet), dtype=int)
    arr_tlp = np.stack((X_dangxet, X_tl, stt), axis=1)

    return arr_tlp

In [38]:
def Gmean(y_test,y_pred):
    cm_WSVM = metrics.confusion_matrix(y_test, y_pred)
    sensitivity = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    specificity = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(sensitivity*specificity)
    return specificity,sensitivity,gmean

In [39]:
def metr(X_train,y_test,test_pred,se,sp,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    print("So luong samples: ",len(X_train))
    print("\n",classification_report(y_test, test_pred))
    print("SP      : ",sp)
    print("SE      : ",se)
    print("Gmean   : ",gmean)
    print("F1 Score: ",f1_score(y_test, test_pred))
    print("Accuracy: ",accuracy_score(y_test,test_pred))
    print("AUC     : ",roc_auc_score(y_test, test_pred))
    print("Ma tran nham lan: \n",confusion_matrix(y_test, test_pred))
    

In [40]:
def metr_text(f,X_train,y_test,test_pred,sp,se,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    f.write(f"\n\nSo luong samples Tong: {len(X_train)+len(y_test)}")
    f.write(f"\n\nSo luong samples training: {len(X_train)}")
    f.write(f"\nSo luong samples testing: {len(y_test)}\n")
    f.write("\n"+str(classification_report(y_test, test_pred)))
    f.write(f"\nSP      : {sp:0.4f}")
    f.write(f"\nSE      : {se:0.4f}")
    f.write(f"\nGmean   : {gmean:0.4f}")
    f.write(f"\nF1 Score: {f1_score(y_test, test_pred):0.4f}")
    f.write(f"\nAccuracy: {accuracy_score(y_test,test_pred):0.4f}")
    f.write(f"\nAUC     : {roc_auc_score(y_test, test_pred):0.4f}")
    f.write("\n\nMa tran nham lan: \n"+str(confusion_matrix(y_test, test_pred)))

In [41]:
def compute_weight(X, y,name_method ="actual_hyper_lin", name_function = "exp", beta = None,C = None, gamma = None, u = None, sigma = None):
    method = fuzzy.method()
    function = fuzzy.function()
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    try:
        if name_method == "own_class_center": 
            d = method.own_class_center(X, y)
        elif name_method == "estimated_hyper_lin": # actual_hyper_lin, own_class_center
            d = method.estimated_hyper_lin(X, y)
        elif name_method == "own_class_center_opposite":
            d = method.own_class_center_opposite(X, y)
        elif name_method == 'actual_hyper_lin':
            d = method.actual_hyper_lin(X, y,C = C, gamma = gamma)
        elif name_method == 'own_class_center_divided':
            d = method.own_class_center_divided(X, y)
        elif name_method == "distance_center_own_opposite_tam":
            d_own, d_opp, d_tam = method.distance_center_own_opposite_tam(X,y)
        else:
            print('dont exist method')
        
        if name_function == "lin":
            W = function.lin(d)
        elif name_function == "exp":
            W = function.exp(d, beta)
        elif name_function == "lin_center_own":
            W = function.lin_center_own(d, pos_index,neg_index)
        elif name_function == 'gau':
            W = function.gau(d, u, sigma)
        elif name_function == "func_own_opp_new":
            W = function.func_own_opp_new(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v1":
            W = function.func_own_opp_new_v1(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v2":
            W = function.func_own_opp_new_v2(d_own,d_opp,pos_index,neg_index,d_tam)
    except Exception as e:
        print('dont exist function')
        print(e)
    # pos_index = np.where(y == 1)[0]
    # neg_index = np.where(y == -1)[0]
    r_pos = 1
    r_neg = len(pos_index)/len(neg_index)
    m = []
    W = np.array(W)
    m = W[pos_index]*r_pos
    m = np.append(m, W[neg_index]*r_neg)
    return m

In [42]:
def fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction):
    if namemethod =="own_class_center_opposite" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_center)
    elif namemethod =="own_class_center" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="own_class_center_divided" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="estimated_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="actual_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_actual)
    else:   
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction)
    return distribution_weight

In [43]:
def data_tomelinks_final(C,weight,X_test,y_test,X_train,y_train,n_neighbors,arr_tlp,clf=None,namemethod=None,namefunction=None):
    ro1 = 0.1
    ro3 = 0.1
    ro4 = 0.5
    ro2 = 0.5
    
    new_W = weight
    pos_index = np.where(y_train == 1)[0]
    neg_index = np.where(y_train == -1)[0]
    clf = Wsvm(C,new_W)
    clf.fit(X_train, y_train)
    # y_predict = clf.predict(X_test)
    # specificity,sensitivity,gmean = Gmean(y_test,y_predict)
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train)

    # # Mẫu âm bị phân loại sai -> Giảm trọng số của mẫu âm đó
    # neg_pred = clf.predict(X_train[neg_index])
    # idx_neg_wrong = np.where(neg_pred != -1.0)
    # new_W[idx_neg_wrong] =  new_W[idx_neg_wrong]*0.5 # giam manh (Co-Author*0.9)

    # Tăng, giảm trọng số của các mẫu trong TLPs
    # Trường hợp 1, 2, 3, 4
    ind_nn_pos = [] # chứa chỉ số của các mẫu dương bị phân loại sai trong ind_posX
    y_nn_pos = [] # chứa nhãn của k mẫu dữ liệu gần nhất với mẫu dương được xét
    ind_nn_neg = [] # chứa chỉ số của các mẫu âm bị phân loại sai trong ind_negX
    y_nn_neg = [] # chứa nhãn của k mẫu dữ liệu gần nhất với mẫu âm được xét

    for ind,i in enumerate(arr_tlp):
        y_pred_pos = clf.predict([X_train[i[0]]]) #positive
        y_pred_neg = clf.predict([X_train[i[1]]]) #negative
        if (y_pred_pos == 1) and (y_pred_neg == 1): #dương được dự đoán đúng & âm bị dự đoán sai (TH1)
            new_W[i[0]] = new_W[i[0]]*(1 + ro1) 
            new_W[i[1]] = new_W[i[1]]*(1 - ro1)
            i[2] = 1

            ind_nn_neg.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[i[1]]])[1]).tolist()  
            for j in knn_X[0]:
                y_nn_neg.append(y_train[j])   # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn_neg

        if (y_pred_pos == -1) and (y_pred_neg == -1): #âm được dự đoán đúng & dương bị dự đoán sai (TH3)
            new_W[i[0]] = new_W[i[0]]*(1 + ro3) 
            new_W[i[1]] = new_W[i[1]]*(1 - ro3)
            i[2] = 3

            ind_nn_pos.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[i[0]]])[1]).tolist()  
            for j in knn_X[0]:
                y_nn_pos.append(y_train[j])   # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn_pos

    ind_nn_neg = np.array(ind_nn_neg)
    y_nn_neg = np.array(y_nn_neg)
    if len(y_nn_neg)>0: #(TH2)
        y_nn_neg = np.array_split(y_nn_neg, len(y_nn_neg)/n_neighbors) 
        for ind1,id1 in enumerate(ind_nn_neg): 
            if -1 not in y_nn_neg[ind1][1:]:      # Nếu không có nhãn -1 xung quanh X_train[i] bị dự đoán sai => nhiễu âm -> giảm mạnh trọng số
                for a in arr_tlp[[id1]]:
                    new_W[a[1]] = new_W[a[1]]*ro2
                arr_tlp[id1][2] = 2


    ind_nn_pos = np.array(ind_nn_pos)
    y_nn_pos = np.array(y_nn_pos)
    if len(y_nn_pos)>0: #(TH4)
        y_nn_pos = np.array_split(y_nn_pos, len(y_nn_pos)/n_neighbors) 
        for ind2,id2 in enumerate(ind_nn_pos):   
            if 1 not in y_nn_pos[ind2][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu dương -> giảm mạnh trọng số
                for a in arr_tlp[[id2]]:
                    new_W[a[0]] = new_W[a[0]]*ro4
                arr_tlp[id2][2] = 4

    return new_W

In [44]:
######################################## Both the membership function & AFW-CIL proposed without Change rate for Yeast dataset ##############################
C = 100
T = 5 # Số vòng lặp
N = 1 # Số lần bốc
n_neighbor = 6

# data = [Co_Author, Abanole, Ecoli, Ecloli1, Ecoli3, Glass1, Glass4, Haberman, Waveform, New_thyroid2, Page_blocks,
#             Pima_Indians_Diabetes, Satimage, Transfusion, Yeast]

#Haberman dataset
# dataset = Haberman_KFold
# beta_center, beta_estimate, beta_actual = 1, 1, 0.6 # !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!

#Ecoli dataset
dataset = Ecoli_Kfold
beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7

# dataset = Pima
# beta_center, beta_estimate, beta_actual = 0.5, 1, 0.5

# dataset = Transfution_Kfold
# beta_center, beta_estimate, beta_actual = 0.5, 0.8, 0.1

# dataset = Yeast_KFold
# beta_center, beta_estimate, beta_actual = 0.9, 0.2, 0.3

# from Processing_Data import Co_Author_50_250_ir, Co_Author_200_1000_ir, Co_Author_100_500_ir, Co_Author_50_250, Co_Author_250_750, Co_Author_100_900, Co_Author_100_700, Co_Author_100_500, Co_Author_50_350, Co_Author_200_1000, Co_Author_200_1000_1, Co_Author_300_1500
# dataset = Co_Author_100_500_ir
# beta_center, beta_estimate, beta_actual = 0.5, 0.8, 0.1

name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]

time = datetime.now().strftime("%d%m%Y_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]
filepath = f'./Experiment/Data_{filename}_Full.csv'

#svc lib
svc = SVC(probability=True, kernel='linear')
#svm scratch
svm_scr = Svm(C)
#W.svm
for n in range(0,N):
    print("Lan boc: ",n+1)
    X, y = dataset.load_data()
    print(X.shape)
    kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
    header = ['Time','Fold','T','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
    data = []
    with open(f'./Experiment/Data_{filename}_{time}_Full.csv', 'a', encoding='UTF8', newline='') as f3:
        writer = csv.writer(f3)
        writer.writerow(header)
        fold = 1
        for train_index, test_index in kfold_validation.split(X,y):   
            print("\nFold thu ",fold)            
            # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
            # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]
            print(X_test.shape)
            
            #Scalling Data
            sc_X = StandardScaler()
            X_train = sc_X.fit_transform(X_train)
            X_test = sc_X.transform(X_test)
            y_train = np.array(y_train)

            #################################### NOT USING AFW-CIL #####################################

            #Svm library
            print("SVM LIBRARY starting...\n")
            test_pred = svm_lib(X_train, y_train,X_test)
            sp,se,gmean = Gmean(y_test,test_pred)
            name1 = 'SVM'
            name2 = 'SVM'
            data.append([n+1,fold,1,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            
            #Wsvm
            print("W.SVM starting...\n")
            N, d = X_train.shape
            distribution_weight = np.ones(N)
            test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
            sp,se,gmean = Gmean(y_test,test_pred)
            name1 = 'WSVM'
            name2 = 'WSVM'
            data.append([n+1,fold,1,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

            # #FuzyyWsvm
            # for namemethod in name_method:
            #     for namefunction in name_function:
            #         if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
            #             continue
            #         elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
            #             continue
            #         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
            #             continue
            #         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
            #             continue
            #         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
            #             continue
            #         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
            #             continue
            #         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
            #             continue
            #         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
            #             continue
            #         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
            #             continue
            #         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
            #             continue
            #         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
            #             continue
            #         else:
            #             print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
            #             distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
            #             __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
            #             test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
            #             sp,se,gmean = Gmean(y_test,test_pred)
            #             data.append([n+1,fold,1,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            

            ########################################## USING AFW-CIL #############################################
            print("\n------------- Using AFW-CIL---------------------\n")
            arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
            #FuzyyWsvm
            for namemethod in name_method:
                for namefunction in name_function:
                    if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                        continue
                    elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    else:
                        
                        print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                        distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)                            
                        for i in range(0,T):
                            # new_W = lfb(f,C,distribution_weight,namemethod,namefunction,T,X_test,y_test,X_train,y_train,n_neighbor,thamso1,thamso2)
                            new_W = data_tomelinks_final(C,distribution_weight,X_test,y_test,X_train,y_train,n_neighbor,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
                            test_pred = wsvm(C,X_train, y_train, X_test, new_W)
                            sp,se,gmean = Gmean(y_test,test_pred)
                            data.append([n+1,fold,i+1,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
                            print("At T = ", i+1)
            
            fold = fold + 1
        writer.writerows(data)
            


Lan boc:  1
(336, 7)

Fold thu  1
(68, 7)
SVM LIBRARY starting...

W.SVM starting...


------------- Using AFW-CIL---------------------

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.

KeyboardInterrupt: 

In [47]:

C = 100
T = 5 # Số vòng lặp
N = 1 # Số lần bốc
n_neighbor = 6

# data = [Co_Author, Abanole, Ecoli, Ecloli1, Ecoli3, Glass1, Glass4, Haberman, Waveform, New_thyroid2, Page_blocks,
#             Pima_Indians_Diabetes, Satimage, Transfusion, Yeast]

#Haberman dataset
# dataset = Haberman_KFold
# beta_center, beta_estimate, beta_actual = 1, 1, 0.6 # !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!

#Ecoli dataset
dataset = Ecoli_Kfold
beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7
time = datetime.now().strftime("%d%m%Y_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]
for n in range(0,N):
    print("Lan boc: ",n+1)

    kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
    header = ['Time','Fold','T','Name Method']
    data = []
    with open(f'./Experiment/Data_{filename}_{time}_Full.csv', 'a', encoding='UTF8', newline='') as f3:
        writer = csv.writer(f3)
        writer.writerow(header)
        fold = 1
        for train_index, test_index in kfold_validation.split(X,y):   
            print("\nFold thu ",fold)            
            
            name1 = 'SVM'
            name2 = 'SVM'
            data.append([n+1,fold,1,name1])
            

            writer.writerows(data)    
            # fold = fold + 1
            

Lan boc:  1

Fold thu  1

Fold thu  1

Fold thu  1

Fold thu  1

Fold thu  1


In [ ]:
# # ######################################## Both the membership function & AFW-CIL proposed + Change rate for Waveform, Satimage dataset################################
# C = 100
# T = 5
# N = 5
# n_neighbor = 5
# # new_rate = [1/5, 1/7, 1/9, 1/11, 1/15] #for Waveform dataset
# new_rate = [1/11, 1/15] #for Satimage dataset

# # # Waveform dataset
# # dataset = Waveform_KFold
# # beta_center, beta_estimate, beta_actual = 0.3, 0.2, 0.5

# # Satimage dataset
# dataset = Satimage
# beta_center, beta_estimate, beta_actual = 0.9, 0.8, 0.2


# name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
# name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_Full.csv'

# #svc lib
# svc = SVC(probability=True, kernel='linear')
# #svm scratch
# svm_scr = Svm(C)
# #W.svm
# for n in range(0,N):
#     print("Lan boc: ",n+1)
#     for ir in new_rate:
#         X, y = dataset.load_data(ir)
#         print(X.shape)
#         kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
#         header = ['Times','IR','Fold','T','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
#         data = []
#         with open(f'./Experiment/Data_{filename}_{time}_Full.csv', 'a', encoding='UTF8', newline='') as f3:
#             writer = csv.writer(f3)
#             writer.writerow(header)
#             fold = 1
#             for train_index, test_index in kfold_validation.split(X,y):               
#                 # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
#                 # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
#                 X_train, y_train = X[train_index], y[train_index]
#                 X_test, y_test = X[test_index], y[test_index]
#                 print(X_test.shape)
                
#                 #Scalling Data
#                 sc_X = StandardScaler()
#                 X_train = sc_X.fit_transform(X_train)
#                 X_test = sc_X.transform(X_test)
#                 y_train = np.array(y_train)

#                 #################################### NOT USING AFW-CIL #####################################

#                 #Svm library
#                 print("SVM LIBRARY starting...\n")
#                 test_pred = svm_lib(X_train, y_train,X_test)
#                 sp,se,gmean = Gmean(y_test,test_pred)
#                 name1 = 'SVM'
#                 name2 = 'SVM'
#                 data.append([n+1,ir,fold,1,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
                
#                 #Wsvm
#                 print("W.SVM starting...\n")
#                 N, d = X_train.shape
#                 distribution_weight = np.ones(N)
#                 test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                 sp,se,gmean = Gmean(y_test,test_pred)
#                 name1 = 'WSVM'
#                 name2 = 'WSVM'
#                 data.append([n+1,ir,fold,1,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#                 #FuzyyWsvm
#                 for namemethod in name_method:
#                     for namefunction in name_function:
#                         if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                             continue
#                         elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         else:
#                             print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                             distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                             __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                             test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                             sp,se,gmean = Gmean(y_test,test_pred)
#                             data.append([n+1,ir,fold,1,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
                

#                 ########################################## USING AFW-CIL #############################################
#                 print("\n------------- Using AFW-CIL---------------------\n")
#                 arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
#                 #FuzyyWsvm
#                 for namemethod in name_method:
#                     for namefunction in name_function:
#                         if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                             continue
#                         elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         else:
#                             print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                             distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                             __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)                            
#                             for i in range(0,T):
#                                 # new_W = lfb(f,C,distribution_weight,namemethod,namefunction,T,X_test,y_test,X_train,y_train,n_neighbor,thamso1,thamso2)
#                                 new_W = data_tomelinks_final(C,distribution_weight,X_test,y_test,X_train,y_train,n_neighbor,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
#                                 test_pred = wsvm(C,X_train, y_train, X_test, new_W)
#                                 sp,se,gmean = Gmean(y_test,test_pred)
#                                 data.append([n+1,ir,fold,i+1,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#                 fold = fold + 1
#             writer.writerows(data)
